# Loading Portfolio data

The `ppt/portfolio_analysis.py` file contains functions for predefined analyses, however, you can still using the `StockPriceLoader` and `CalculateStockValue` classes directly to load the raw data to do your own independant analysis.

However, everyone's portfolio is unique and everyone has slightly different requirements for their analysis (one of the reasons for making this library!). 

This notebook explains how you can directly use the data loading classes which are wrapped within the main `Portfolio` object for low level access to the raw data.

In [1]:
%load_ext autoreload
%autoreload 2

# !pip install nb_black
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sys

sys.path.append("..")

<IPython.core.display.Javascript object>

In [5]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from utils import print_properties

from ppt.data_loader import (
    BenchMarkLoader,
    CurrencyLoader,
    PositionLoader,
    StockPriceLoader,
)

ImportError: cannot import name 'BenchMarkLoader' from 'ppt.data_loader' (../ppt/data_loader.py)

<IPython.core.display.Javascript object>

## Input data format

See the `README` for explainations on the required data input format.

In [ ]:
# example input
pd.read_csv("../data/raw/purchase_info.csv")

## Load Positions

The base case for all classes is `PositionLoader` this loads the raw input data and extracts some key information which is used by other classes to help with data processing

In [ ]:
# automatically loads from default data location (../data/raw/purchase_info.csv)
positions = PositionLoader()

In [ ]:
# properties
print_properties(positions)

In [ ]:
# dataframe with input csv data
positions.positions

In [ ]:
# tickers
positions.tickers

In [ ]:
# metadata
positions.stock_metadata

## Load Stock Prices

You are able to load a dataframe of daily closing stock prices for all stock tickers present in the `purchase_info.csv`, starting from the earliest date of your first position, using the `StockPriceLoader` class.

Note the default location for the input data file is: `../data/raw/purchase_info.csv`, however, you can specify your own location using the `input_data_source` argument.

The `StockPriceLoader` class uses the `ffn` library to load the stock prices for free from Yahoo ([see ffn library for more details](https://pmorissette.github.io/ffn/)).

In [ ]:
# load stock prices of tickers in your purchase_info csv file
stock_prices = StockPriceLoader()

In [ ]:
# __repr__ gives a list of tickers and the starting date
stock_prices

In [ ]:
print_properties(stock_prices)

The `StockPriceLoader` inherits the same properties from the `PositionLoader` seen above. There are two new properties: `daily_stock_prices_local_currency` and `daily_stock_prices_usd`. These are dataframes containing the daily stock prices for each ticker and the daily stock prices converted from the local currency to usd, starting from the earliest registered date in the input csv.

In [ ]:
# local currencies
stock_prices.daily_stock_prices_local_currency.head(10)

In [ ]:
# converted to USD
stock_prices.daily_stock_prices_usd.head(10)

This is the 'raw' data from which all further calculations are made. You can use these dataframe of stock prices for your own individual stock analysis.

In [ ]:
stock_prices.daily_stock_prices_local_currency["NVDA"].plot()
plt.title("Nvidia stock price")
plt.ylabel("USD")
plt.show()

## Benchmarks

Sometimes you will want to benchmark your portfolio against indices or other instruments.

The `BenchMarkLoader` inherits from `PositionLoader`

A single benchmark passed as a string or a list of benchmarks can be passed to the function.

In [ ]:
# load SNP 500
benchmark = "^GSPC"
bm = BenchMarkLoader(benchmark_tickers=benchmark)
bm.benchmark_stock_prices

In [ ]:
# list benchmarks
bm.benchmarks

## Currency Loader

Historical daily exchange rates are collected from yahoo finance.

In [ ]:
cur = CurrencyLoader()

print_properties(cur)

In [ ]:
cur.currencies

In [ ]:
# daily exchange rates (including USD/USD)
cur.xrates